<a href="https://colab.research.google.com/github/MousamCodes/FirstRepo/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
%pip install kaggle

Upload your kaggle json file

In [41]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Importing dataset from kaggle using **API** takes less time to uploadthe dataset.


In [42]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


Extracting the compressed dataset

In [43]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset,'r' ) as zip:
  zip.extractall()

  print("Dataset is extracted")


Dataset is extracted


Importing the Dependencies

In [44]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
#nltk natural language toolkit for processing textual data
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
#cant feed textual data to model to convert it to numeric data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [45]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
#printing stop words in english , stopwords doesnt add anym meaning to data
#reducing the complexity of data
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

###Data Processing:
  Loading the data from csv file to pandas df
  and encoding it in standard encoding as ISO-8859-1 or latin-1 (same).If the encoding is not specified and it differs from the files actual encoding, the data may be misinterpreted, leading to errors like UnicodeDecodeError or garbled text.

In [47]:
twitter = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='latin-1')

In [48]:
twitter.shape #16lakh tweets and 6 cols

(1599999, 6)

It contains the following 6 fields:

1.target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

2.ids: The id of the tweet ( 2087)

3.date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

4.flag: The query (lyx). If there is no
 query, then this value is NO_QUERY.

5.user: the user that tweeted (robotickilldozr)

6.text: the text of the tweet (Lyx is cool)

In [49]:
twitter.head() #first value is being read as col name

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [50]:
column_names = ['target','ids','date','flag','user','text']
twitter = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='latin-1',names=column_names)


**Problem**: the first data (tweet) is being read as column to fix this. Naming the cols and reading the dataset again

In [51]:
twitter.shape

(1600000, 6)

In [52]:
twitter.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


The tweets have been annotated (0 = negative, 2 = neutral, 4 = positive)
this is provided and can be used to detect the sentiment

In [53]:
#checking the number of missing values in the dataset null=missing values
twitter.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


no missing dataset

##Now:
###Checking the distribution of target columns or how many negative , positive and neutral tweets


In [54]:
twitter['target'].value_counts()

,count
target,
0,800000
4,800000


As the distribuion is eeven we dont need to do upsampling and down sampling

Convert the target
'4' to '1'


In [55]:
twitter.replace({'target':{4:1}}, inplace = True)
 #replace all 4 to 1 and
#inplace means to store this change in original dataframe

In [56]:
twitter['target'].value_counts()

,count
target,
0,800000
1,800000


0 --> Negative Tweet
1 --> Positive Tweet

**STEMMING**

stemming is a natural language processing (NLP) technique used to reduce words to their base or root form. The goal of stemming is to remove suffixes from words, so that different forms of a word are treated as the same word.

For example, words like "playing", "played", and "plays" all stem to the root word "play".

this to reduce the time of process and size , dimension reduction.

In [57]:
 port_stem = PorterStemmer()

In [58]:
 def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) #anything that is not
   # a alphabet (either lowercase or uppercase is removed
  stemmed_content = stemmed_content.lower() #converting uppercase to lowercase
  stemmed_content = stemmed_content.split() #splitting all those words
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  #checing if the processed content have any stopwords if yes no stem if not we stem
  stemmed_content = ' '.join(stemmed_content) #join tweets of words in each tweets

  return stemmed_content

In [59]:
 twitter['stemmed_content'] = twitter['text'].apply(stemming)

In [60]:
twitter.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [61]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [62]:
print(twitter['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [63]:
print(twitter['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


Sperating the data and label

In [64]:
x=twitter['stemmed_content'].values
y=twitter['target'].values

In [65]:
print(x)


['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [66]:
print(y)

[0 0 0 ... 1 1 1]


Splitting the data to the training data and test data
80% training data and 20% test or vice versa if u put test size 0.1

In [67]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [68]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


###TURNING CHARACTERS TO NUMERIC DATA AS MODEL DOESNT UNDERSTAND SHIT

####Using feature extraction or feature engineering

In [69]:
# Assuming 'x' is your original text data
vector = TfidfVectorizer()
x = vector.fit_transform(x)

# Now you can split the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

In [70]:
print(x_train)

  (0, 504044)	0.2725664180116586
  (0, 126118)	0.37484032460318173
  (0, 409438)	0.35861719248416485
  (0, 213554)	0.5289928671316323
  (0, 271144)	0.42037613603065954
  (0, 511398)	0.4472655127083222
  (1, 185304)	1.0
  (2, 472107)	0.15171423197968203
  (2, 470034)	0.18720443014780436
  (2, 126118)	0.458250851237366
  (2, 205312)	0.16175394132247142
  (2, 333015)	0.1678067864881519
  (2, 468994)	0.3209933731315585
  (2, 143635)	0.18906085394224742
  (2, 173898)	0.18807038740989349
  (2, 152669)	0.20261379420870737
  (2, 90030)	0.31341180319788786
  (2, 307827)	0.24104556465616028
  (2, 511810)	0.3361762065862128
  (2, 149331)	0.29041017783193285
  (2, 500375)	0.3297483694639192
  (3, 175126)	0.27847482359309217
  (3, 474893)	0.27096057778571386
  (3, 198842)	0.3743117172596608
  (3, 468994)	0.2901937126965763
  :	:
  (1279996, 367538)	0.21106719280315753
  (1279996, 274436)	0.22225898756809326
  (1279996, 500911)	0.27035064240730977
  (1279996, 430827)	0.3500359054978762
  (1279996, 2

In [71]:
print(x_test)

  (0, 472107)	0.3147167883759488
  (0, 152729)	0.25558796449659876
  (0, 322141)	0.17845040984735672
  (0, 35963)	0.1625503234745387
  (0, 17371)	0.17202593770788552
  (0, 159349)	0.23667287956118194
  (0, 78282)	0.26825866789191927
  (0, 485830)	0.17939145201290538
  (0, 197646)	0.28105531331632916
  (0, 448260)	0.22002407032145396
  (0, 122384)	0.3647011401945221
  (0, 460402)	0.34917937451757414
  (0, 312772)	0.4530719148789886
  (1, 296228)	0.23864015910121067
  (1, 422904)	0.20442035768338634
  (1, 17371)	0.17542242601230812
  (1, 7432)	0.2554738596093585
  (1, 402004)	0.3950023328793501
  (1, 250883)	0.3354843401454554
  (1, 167742)	0.4782713992526613
  (1, 406721)	0.5548902184495058
  (2, 39714)	0.38049318170650237
  (2, 296292)	0.25710183920061486
  (2, 103267)	0.36475227351737843
  (2, 462385)	0.4334875869226873
  :	:
  (319994, 192710)	0.5354792173860652
  (319995, 374661)	0.36005895413850625
  (319995, 480353)	0.2380021572686678
  (319995, 268654)	0.2574297916665581
  (31999

Training the Machine Learning Model

Logistic  Regression using Python , using in built functions a dependent variable and one or more independent variables. It’s commonly used in predictive analytics and machine learning to find patterns in data.

In [72]:
model = LogisticRegression(max_iter=1000)

In [73]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

*MODEL EVALUATION*

In [78]:
#accuracy score on the training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(y_train, x_train_prediction) #between prediction and target

In [79]:
print('Accuracy score on the training data :',training_data_accuracy)

Accuracy score on the training data : 0.8047734375


0.80 means 80% accurate, model will make correct prediction for 80 tweets out of 100.

###Now accuracy score of test data

In [80]:
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(y_test, x_test_prediction)

In [81]:
print('Accuracy score on the test data :',test_data_accuracy)

Accuracy score on the test data : 0.777475


70 % correct

overfitting

So we can conclude that 70% 80% is close to each other so the model has performed well.

save the trained model

In [82]:
import pickle

In [83]:
filename ='trained_model.sav'
pickle.dump(model, open(filename,'wb'))

Using the model to predict others

In [84]:
#load the save model
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

In [88]:
x_new= x_test[200]
print(y_test[200])

prediction = loaded_model.predict(x_new)
print(prediction)

if (prediction[0] ==0):
  print("negative tweet")

else:
  print("positive tweet")

1
[1]
positive tweet


In [89]:
x_new= x_test[3]
print(y_test[3])

prediction = loaded_model.predict(x_new)
print(prediction)

if (prediction[0] ==0):
  print("negative tweet")

else:
  print("positive tweet")

0
[0]
negative tweet
